In [25]:
import openmeteo_requests
import time
import numpy as np
import openmeteo_requests
import requests
from tqdm import tqdm
import pandas as pd
from datetime import datetime


# What Are We Looking For?  
We are looking for live data collected by the ECMWF, which provides global measurements on a 0.25-degree resolution grid. This dataset is ideal due to its extensive historical records (ERA5) and our team’s familiarity with it. Eric’s prior experience with this dataset at Cambridge further adds to its convenience and suitability for our work.  

## ECMWF API  
At first glance, accessing ECMWF’s real-time data requires authorization. Additionally, many public APIs leverage this data source to provide their services. Therefore, our initial step will be to explore these public APIs before investigating direct access to ECMWF’s API.  

### Ambiguity in Access  
Upon reviewing some public APIs and revisiting the ECMWF documentation, there appears to be ambiguity regarding access:  
- Real-time data is available to authorized or licensed users, either directly from ECMWF or through a Member or Co-operating State National Meteorological Service. This is determined during the ordering process.  
- Open real-time data is reportedly accessible free of charge via the public FTP.  

While the documentation suggests that free access is available, it’s unclear whether this applies universally or only to authorized parties. Further clarification is needed to understand the terms of access.  


In [15]:
START_DATE = "2024-01-01"
END_DATE = "2024-01-20"
VAR = "temperature_2m"
LAT_RANGE = np.linspace(-90, 90, 721)
LON_GROUPS = np.split(np.linspace(-180, 179.75, 1440), 8)

In [24]:

from ecmwf.opendata import Client
client = Client("ecmwf", beta=True)
parameters = ['10u', '10v','2t']
filename = 'aifs_medium-2t-wind.grib'

client.retrieve(
    date=0,
    time=0,
    step=12,
    stream="oper",
    type="fc",
    levtype="sfc",
    model="aifs",
    param=parameters,
    target=filename
)

20250123000000-12h-oper-fc.grib2:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

# open-meteo forecast
This is said on there website:
"By default, we provide forecasts for 7 days, but you can access forecasts for up to 16 days. If you're interested in past weather data, you can use the Past Days feature to access archived forecasts."

So the "past days" in the parameters below is the forecast of the past days. Although this is not interesting to send to miners, it could be used to calculated the Z value in the reward function based on the error of this API. 

In [16]:
om = openmeteo_requests.Client()
url_forecast = "https://api.open-meteo.com/v1/forecast"

forecast_data = []
for lat in tqdm(LAT_RANGE):
	fixed_lat_data = []
	for lon_group in LON_GROUPS:
		print("yes")
		params = {
			"latitude": [lat] * len(lon_group),
			"longitude": lon_group,
			"hourly": [VAR],
			"models": "ecmwf_ifs025",
			"start_date": START_DATE,
			"end_date": END_DATE,
		}
		responses = om.weather_api(url_forecast, params=params)
		fixed_lat_data.extend([c.Hourly().Variables(0).ValuesAsNumpy() for c in responses])
		time.sleep(0.1)

	fixed_lat_data = np.stack(fixed_lat_data)
	forecast_data.append(fixed_lat_data)

forecast_data = np.stack(forecast_data)

# (lat, lon, time)
print(forecast_data.shape)

  0%|          | 0/721 [00:00<?, ?it/s]

yes
yes
yes


  0%|          | 0/721 [00:00<?, ?it/s]

yes
yes


OpenMeteoRequestsError: {'error': True, 'reason': 'Minutely API request limit exceeded. Please try again in one minute.'}

In [ ]:
np.save("forecast_data.npy", forecast_data)


---------------

# open-meteo archive

the code below shows that we can only get the tempratures of "yesterday", so you will have all the data at one point in time.
Do note that on their website they claim the following: <br>
"You can access past weather data dating back to 1940. However, **there is a 5-day delay in the data**. If you want information for the most recent days, you can use the forecast API and adjust the Past Days setting."

When calling the API **we see a max 24hrs delay**, but I'm not sure if we can trust this observation. 

In [66]:
om = openmeteo_requests.Client()
url_archive = "https://archive-api.open-meteo.com/v1/archive"

archive_data = []
for lat in LAT_RANGE:
	fixed_lat_data = []
	for lon_group in LON_GROUPS:
		params = {
			"latitude": [lat] * len(lon_group),
			"longitude": lon_group,
			"hourly": [VAR],
			"start_date": START_DATE,
			"end_date": END_DATE,
		}
		responses = om.weather_api(url_archive, params=params)
		fixed_lat_data.extend([c.Hourly().Variables(0).ValuesAsNumpy() for c in responses])

	fixed_lat_data = np.stack(fixed_lat_data)
	archive_data.append(fixed_lat_data)

archive_data = np.stack(archive_data)

# (lat, lon, time)
print(archive_data.shape)

array([-29.25, -29.6 , -29.55, -29.55, -29.35, -29.35, -29.3 , -29.15,
       -29.15, -28.85, -28.7 , -28.25, -27.45, -26.9 , -26.15, -25.45,
       -24.95, -24.3 , -23.65, -22.9 , -22.25, -21.9 , -21.85, -21.85,
       -21.65, -20.85, -20.5 , -20.  , -19.6 , -19.3 , -19.2 , -19.25,
       -19.45, -19.6 , -19.7 , -19.8 , -19.95, -20.2 , -20.4 , -20.55,
       -20.65, -20.6 , -20.55, -20.5 , -20.45, -20.45, -20.5 , -20.55,
       -20.5 , -20.5 , -20.75, -20.95, -21.05, -20.95, -20.8 , -20.6 ,
       -20.35, -20.3 , -20.35, -20.4 , -20.45, -20.95, -21.15, -21.35,
       -21.65, -21.85, -22.  , -22.2 , -22.45, -22.65, -22.8 , -22.9 ,
       -23.  , -23.1 , -22.85, -22.8 , -22.35, -21.95, -21.85, -21.95,
       -21.8 , -21.45, -21.25, -21.1 , -20.85, -21.15, -21.  , -20.85,
       -20.75, -20.75, -21.  , -21.6 , -21.95, -22.15, -22.5 , -22.85,
       -23.25, -22.7 , -23.1 , -23.45, -23.95, -24.25, -24.3 , -24.4 ,
       -24.55, -24.75, -25.  , -25.2 , -25.3 , -25.8 , -25.95, -26.  ,
      

In [ ]:
np.save("archive_data.npy", archive_data)

# Calling our miners as an API

In [ ]:
response = requests.post(
    "http://localhost:10913/query",
    headers={"authorization": "whatever"},
    json={
        "lat_start": 51.0,
        "lat_end": 53.0,
        "lon_start": 3.0,
        "lon_end": 7.0,
        "start_time": "2024-10-02T00:00:00",
        "end_time": "2024-10-04T00:00:00",
        "predict_hours": 24,
    }
)

In [ ]:
torch.tensor(response.json()["prediction"])